Thakaa center challenge

In [1]:
path1 = "Workable JD/workable.csv" #path to read JD from file Workable JD 
data =[]

In [2]:
with open(path1) as f:
    for line in f:
        linesplit =line.split(",")
        
        data.append(linesplit)

In [3]:
print (data[0])

['\ufeffNo', 'Category', 'Job', 'What is', 'what does', 'Responsibilities description', 'Job brief', 'Responsibilities', 'Requirements\n']


In [4]:
path2 = "Open source global resumes"
import sys,os,io
resumes=[]  # resumes[[ID][Source][Path][file_name][Extension][Resume_Object]]
Directories =[]
path= './' + path2



In [5]:
# resumes[[ID][Source][Path][file_name][Extension][Resume_String]{Resume_dict}]
def fileExtension(file_name):
    if 'pdf' in file_name:
        return 'pdf'
    elif 'docx' in file_name:
        return 'docx'
    elif 'doc' in file_name:
        return 'doc'
    else:
        return 'unknown'
ID = 0
for root, subFolders, files in os.walk(path ):
    for file_name in files:
        
        Source = 'Open source global resumes'
        Resume_String=''
        Resume_dict= dict()
        resume = [ID,Source,root,file_name,fileExtension(file_name),Resume_String,Resume_dict]
        ID = ID + 1
        resumes.append(resume)

In [6]:
print (resumes[1][2])
print ("OK")


./Open source global resumes
OK


In [7]:
#from docx import Document
#from docx.shared import Inches
#import PyPDF2 as PY

In [8]:
import docx
import PyPDF2

# In this function: read each docx file and convert to a string 
def docx_to_text(resume):
    # print (resume[2]+'/'+resume[3] )
    doc = docx.Document(resume[2]+'/'+resume[3])
    all_paras = doc.paragraphs
    text = ''
    for para in all_paras:
        text =text + str (para.text)
    return text

# In this function: read each PDF file and convert to a string 
def pdf_to_text(resume):
    sample= resume[2]+'/'+resume[3]
    #print (sample)
    pdf_file = open(sample, 'rb')
    read_pdf = PyPDF2.PdfFileReader(pdf_file)
    number_of_pages = read_pdf.getNumPages()
    resumeString=''
    for i in range (0,number_of_pages):
        page = read_pdf.getPage(i)
        page_content = page.extractText()
        for i in page_content:
            resumeString= resumeString + i
    return resumeString

for i in resumes:
    if i[4] == 'docx':
        text = docx_to_text(i)
        i[5]= text
    if i[4] == 'pdf':
        text=''
        try:
            text = pdf_to_text(i)
        except :
            #sample= i[2]+'/'+i[3]
            # text ="Unreadable file" 
            pass

print ("OK")


OK


In [9]:
# resumes[[ID][Source][Path][file_name][Extension][Resume_String]{Resume_dict}]

#/home/mandub/Desktop/Thakaa/Resumes-Filtering
#./Open source global resumes
#!mkdir converted
from pdf2image import convert_from_path
# I call this function once only
def pdf_to_image(pdf_Path,file_name):
    root_path = '/home/mandub/Desktop/Thakaa/Resumes-Filtering'
    file_path=root_path+pdf_Path[1:]+'/'+file_name
    pages = convert_from_path(file_path, dpi=200)
    for idx,page in enumerate(pages):
        #file_name.replace('.', '')
        page.save('converted/'+str(file_name)+str(idx)+'.jpg', 'JPEG')
    print (file_path)
    
pdf_to_image(resumes[1][2],resumes[1][3])    
counter=0
for i in resumes:
    if i[5] == '':
        if i[4]=='pdf':
            counter=counter+1
            #pdf_to_image(i[2],i[3])
        
print (counter)
print (len(resumes))



        
        



/home/mandub/Desktop/Thakaa/Resumes-Filtering/Open source global resumes/4Dickson.pdf
1035
2170


In [1]:
pdf_imge_files = !ls -d '/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/'*.jpg

print (pdf_imge_files)
print ('ok')

['/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/10Jonas.pdf0.jpg', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/10Jonas.pdf1.jpg', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/10Jonas.pdf2.jpg', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/10Jonas.pdf3.jpg', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/1Amy.pdf0.jpg', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/1Amy.pdf1.jpg', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/2Ben.pdf0.jpg', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/2Ben.pdf1.jpg', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/3Carrie.pdf0.jpg', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/3Carrie.pdf1.jpg', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/3M_Polo Yu_Finance Manager.pdf0.jpg', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/3M_Polo Yu_Finance Manager.pdf1.jpg', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/4Dic

In [4]:
import os,io
from google.cloud import vision
import pandas as pd
import pickle
#pickle.dump( resumes, open( "all_resumes.p", "wb" ) )
resumes = pickle.load( open( "all_resumes.p", "rb" ) )
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'Mygooglecloudkey.json'
client = vision.ImageAnnotatorClient()

image_path = '/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/1Amy.pdf0.jpg'

def Image2text(image_path):
    with io.open(image_path, 'rb') as image_file:
        content = image_file.read()
    image = vision.types.Image(content=content)
    response = client.text_detection(image=image)
    df = pd.DataFrame(columns=['locale', 'description'])

    texts = response.text_annotations

    for text in texts:
        df = df.append(
            dict(
                locale=text.locale,
                description=text.description
            ),
            ignore_index=True
        )
    #print(df['description'][0])
    return str(df['description'][0])

#/home/mandub/Desktop/Thakaa/Resumes-Filtering/pickle_files

alist = pdf_imge_files

alist_length= len (alist)
pickle_files = !ls -d '/home/mandub/Desktop/Thakaa/Resumes-Filtering/pickle_files/'*.p
print (pickle_files)
last=[]
for i in pickle_files:
    last.append(int (i.split('/')[-1].replace(".p",'')))
read_next = max(last)
print (read_next)
for index,item in enumerate(alist):
    if (read_next < index):
        try:
            resumetextstring = Image2text(item)
            pickle.dump( [item,resumetextstring], open( "pickle_files/"+ str(index)+".p", "wb" ) )
            print (index)
        except :
            print ("unreadable file"+ item)
            #475  unreadable file/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/Citi_Ivan Leong_VP Inv Advisory.pdf1.jpg
            #1837 unreadable file/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/Sesotec_Aderine Pong_FC.pdf7.jpg
            #1237 unreadable file/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/Lion Global_Joshua Wee_ Accounting Associate.pdf3.jpg
            #1218 unreadable file/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/Legacy Advisors_Annie Wong_Inv Ops Associate.pdf2.jpg
            #953  unreadable file/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/HMGC_Smriti Gurung_Accountant.pdf1.jpg
            #896  unreadable file/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/Graticule AM_Stephen Stoyka_VP.pdf1.jpg
            #833  unreadable file/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/Gateway Partners_Won Seuk Jang_VP.pdf1.jpg
            pass



#resumetextstring = Image2text(pdf_imge_files[0])
#pickle.dump( [pdf_imge_files[0],resumetextstring], open( "pickle_files/"+ str(0)+".p", "wb" ))
#resumetextstring = pickle.load( open( "pickle_files/0.p", "rb" ) )
#print (resumetextstring)

#[str (content),Image2text(content)]
#print (len(resumes))
#print (len(pdf_imge_files))
#pickle.dump( Pdf_imges_to_text, open( "Pdf_imges.p", "wb" ) )
#pickle.dump( resumes, open( "all_resumes.p", "wb" ) )

['/home/mandub/Desktop/Thakaa/Resumes-Filtering/pickle_files/0.p', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/pickle_files/100.p', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/pickle_files/101.p', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/pickle_files/102.p', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/pickle_files/103.p', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/pickle_files/104.p', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/pickle_files/105.p', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/pickle_files/106.p', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/pickle_files/107.p', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/pickle_files/108.p', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/pickle_files/109.p', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/pickle_files/10.p', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/pickle_files/110.p', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/pickle_files/111.p', '/home/mandub/Desktop/Thakaa/Resumes-Filtering/pic

unreadable file/home/mandub/Desktop/Thakaa/Resumes-Filtering/converted/Citi_Ivan Leong_VP Inv Advisory.pdf1.jpg
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698


2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212
2213
2214
2215
2216
2217
2218
2219
2220
2221
2222
2223
2224
2225
2226
2227
2228
2229
2230
2231
2232
2233
2234
2235
2236
2237
2238
2239
2240
2241
2242
2243
2244
2245
2246
2247
2248
2249
2250
2251
2252
2253
2254
2255
2256
2257


ok


In [17]:
import nltk
import numpy
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
stop_words = set (stopwords.words("english"))

def Resumes_words_sents_tokenize_filter(resume):
    words = word_tokenize(resume)
    filter_resume = [w for w in words if not w in stop_words]
    Astring = ''
    for i in filter_resume:
        Astring= Astring+' '+ str(i)
    
    #print (len (sent_tokenize(Astring)))
    return Astring # return filter_resume_string
    
#Resumes_words_sents_tokenize_filter(resumes[3][5])
#print (Astring)
#print (resumes[5][5])


1686
2170
